# Mancala: Player vs AI
In this project we have implemented a playable version of mancala using python with **slight rule tweaks**. We then proceeded to go try and test a few different types of AI to use as an opposing "computer" player for the humans to play against. These strategies were tested in this order:
1. Random moves.
2. Minimax
3. Minimax with alpha beta pruning

Our goal is to produce an AI that plays signficantly better than the random AI while running in real time without the user or player having to wait for long periods of time while the AI computes.
## Rule changes to our mancala
Our game of Mancala plays much like the normal version of mancala. We have varying sized boards that don't impact gameplay, but our main tweak to the original Mancala is:
1. If your last stone for your turn lands in your Mancala, you don't get to make another move. In other words, the turn always alternates every time a player makes a move without exception.
2. When a player has no stones left in any of the pits the game ends and the **none** of the remaining stones on the board are distributed to either players' Mancala pit.
## How to play
In the cell below we define a series of variables with comments explaining what they do. You can simply toggle on your preferred variables and then run the code. If a player is playing, you will have to send your moves through the CLI by typing a valid number seen in the printed Mancala board.
## Options: Toggles with explanations to alter code

The following are options to play out a single game using different players and see the results printed out. Note that enabling or disabling these does not disable the mass-game simulation and statistics.

In [ ]:
MOVE_COUNT = 100 # number of moves (both players combined) before the game configured below end automatically
PLAY_USER_VS_RANDOM = False # whether or not to play the game using input from the terminal  and print out all the moves
PLAY_MINIMAX_VS_RANDOM = False # whether or not to play the minimax version using subclass of Game and print out all the moves
PLAY_ALPHABETA_VS_RANDOM = False # whether or not to play the alphabeta version using subclass of Game and print out all the moves

These are just some misc options that could be useful for simulation testing

In [ ]:
NUMBER_OF_GAMES_TO_SIMULATE = 1000 # number of games to simulate when testing the AI vs random games
USE_SET_RANDOM_SEED = False # whether or not to use a set random seed for the random player
RANDOM_SEED = 109 # the random seed to use if USE_SET_RANDOM_SEED is True

# Libraries used
Our libraries used are imported below. Most importantly, we are using the AIMA library game abstract class such that we will also have access to their minimax and alpha beta pruning algorithms.

In [ ]:
import random
from random import seed
from random import randint
if USE_SET_RANDOM_SEED:
    random.seed(RANDOM_SEED)

In [ ]:
from games4e import *
from utils4e import *

# Implementation
Below is where we implement our logic. We begin by subclassing the 

## Subclassing AIMA Game for Mancala
Below is where we leverage the AIMA library. By subclassing their game class with proper methods, we can then apply minimax and alpha beta pruning to our AI player to improve it's effectiveness.

In [ ]:
# GameState contains the current state of the game
from collections import namedtuple
import random
import sys

# Update GameState to use current_player instead of to_move
GameState = namedtuple('GameState', ['to_move', 'utility', 'board', 'score'])

class Mancala(Game):
    """
    pits_per_player: Number of pits for each player
    stones_per_pit: Number of stones in each pit at the start of the game
    board_size: Total number of pits on the board
    initial: Initial state of the game
    p1_store: Index of player 1's store
    p2_store: Index of player 2's store
    p1_pits_range: Range of indices for player 1's pits
    p2_pits_range: Range of indices for player 2's pits
    """ 
    def __init__(self, pits_per_player=3, stones_per_pit=2):
        self.pits_per_player = pits_per_player
        self.stones_per_pit = stones_per_pit
        
        # Calculate total board size: pits for both players + 2 stores
        self.board_size = 2 * pits_per_player + 2
        
        # Initialize board with specified stones in each pit
        board = [stones_per_pit] * self.board_size
        board[pits_per_player] = 0  # Player 1's store
        board[-1] = 0  # Player 2's store
        
        self.initial = GameState(to_move='1', utility=0, 
                               board=board, score={'1': 0, '2': 0})
        
        # Store indices for easy reference
        self.p1_store = pits_per_player
        self.p2_store = self.board_size - 1
        
        # Store pit ranges for display
        self.p1_pits_range = (0, pits_per_player - 1)
        self.p2_pits_range = (pits_per_player + 1, self.board_size - 2)

    def actions(self, state):
        """Legal moves are the indices of non-empty pits on the current player's side."""
        if state.to_move == '1':
            return [i for i in range(self.pits_per_player) if state.board[i] > 0]
        else:
            return [i for i in range(self.p1_store + 1, self.p2_store) 
                   if state.board[i] > 0]

    def result(self, state, move):
        """Return the state that results from making a move."""
        if move not in self.actions(state):
            return state  # Illegal move has no effect
            
        board = state.board.copy()
        score = state.score.copy()
        to_move = state.to_move
        
        # Pick up stones
        stones = board[move]
        board[move] = 0
        
        # Distribute stones
        current_pit = move
        
        while stones > 0:
            current_pit = (current_pit + 1) % self.board_size
            # Skip opponent's store
            if (to_move == '1' and current_pit == self.p2_store) or \
               (to_move == '2' and current_pit == self.p1_store):
                continue
                
            board[current_pit] += 1
            stones -= 1
            
            # Check for capture on last stone
            if stones == 0:
                if board[current_pit] == 1:
                    # Check if last stone landed in an empty pit on player's side
                    is_p1_side = current_pit < self.p1_store
                    is_p2_side = self.p1_store < current_pit < self.p2_store
                    
                    if (to_move == '1' and is_p1_side) or \
                       (to_move == '2' and is_p2_side):
                        opposite = (2 * self.pits_per_player) - current_pit
                        if board[opposite] > 0:
                            store = self.p1_store if to_move == '1' else self.p2_store
                            board[store] += board[opposite] + 1
                            board[opposite] = 0
                            board[current_pit] = 0
        
        # Update scores
        score['1'] = board[self.p1_store]
        score['2'] = board[self.p2_store]
        
        # Switch to other player
        next_player = '2' if to_move == '1' else '1'
        
        return GameState(to_move=next_player,
                        utility=self.compute_utility(board, score),
                        board=board,
                        score=score)

    def utility(self, state, player):
        """Return the value to player; 1 for win, -1 for loss, 0 for ongoing."""
        return state.utility if player == '1' else -state.utility

    def terminal_test(self, state):
        """A state is terminal if one player's pits are all empty."""
        player1_empty = all(state.board[i] == 0 
                          for i in range(self.pits_per_player))
        player2_empty = all(state.board[i] == 0 
                          for i in range(self.p1_store + 1, self.p2_store))
        
        next_player = '2' if state.to_move == '1' else '1'
        return player1_empty or player2_empty

    def display(self, state):
        """Display the current state of the board in a user-friendly format."""
        board = state.board
        
        # Extract relevant sections of the board
        p1_pits = board[self.p1_pits_range[0]:self.p1_pits_range[1] + 1]
        p1_mancala = board[self.p1_store]
        p2_pits = board[self.p2_pits_range[0]:self.p2_pits_range[1] + 1]
        p2_mancala = board[self.p2_store]

        # Display board
        print('\nP1               P2')
        print('     ____{}____     '.format(p2_mancala))
        
        for i in range(self.pits_per_player):
            pit_num_1 = i  # Use actual index for player 1
            pit_num_2 = self.p2_pits_range[0] + (self.pits_per_player - 1 - i)  # Calculate actual index for player 2
            
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(
                    pit_num_1, p1_pits[i], p2_pits[-(i+1)], pit_num_2))
            else:    
                print('{} -> | {} | {} | <- {}'.format(
                    pit_num_1, p1_pits[i], p2_pits[-(i+1)], pit_num_2))
            
        print('         {}         '.format(p1_mancala))
        print('Turn: P{}'.format(state.to_move))
        print(f"Scores - P1: {state.score['1']}, P2: {state.score['2']}\n")

    def compute_utility(self, board, score):
        """Compute the utility value of the current state."""
        return board[self.pits_per_player] - board[self.pits_per_player * 2 + 1]
    
    def play_game(self, *players):
        """Play an n-person, move-alternating game."""
        state = self.initial
        while True:
            for player in players:
                move = player(self, state)
                state = self.result(state, move)
                if self.terminal_test(state):
                    self.display(state)
                    # return self.utility(state, self.to_move(self.initial))
                    return state

# Creating player classes
Below we have a general function that takes in two player classes and then plays out a game of mancala between them, returning the result. This can be used to play many games with different types of players in any orientation. The function approach is powerful as it allows us to make player classes once, and then do any permutation of games, such as player vs player, or player vs minimax.

In [ ]:
# Function to play out a game between two player classes passed in as input
def play_mancala(player1, player2):
    """Play a game of Mancala with graceful exit handling."""
    try:
        game = Mancala(pits_per_player=3, stones_per_pit=2)
        
        # Play the game
        final_state = game.play_game(player1, player2)
        
        # Display final score
        print("\nGame Over!")
        print(f"Final Score - {player1.name}: {final_state.score['1']}, "
              f"{player2.name}: {final_state.score['2']}")
        
        # Determine winner
        if final_state.score['1'] > final_state.score['2']:
            print(f"{player1.name} wins!")
            return 1
        elif final_state.score['2'] > final_state.score['1']:
            print(f"{player2.name} wins!")
            return -1
        else:
            print("It's a tie!")
            return 0
        
    except (KeyboardInterrupt, EOFError):
        print("\nGame terminated by user!")
        sys.exit(0)

In [ ]:
class GameExit(Exception):
    """Custom exception for handling game exits."""
    pass

class HumanPlayer:
    """Interactive player that asks for moves through the command line."""
    def __init__(self, name="Human"):
        self.name = name

    def __call__(self, game, state):
        """Make a move by querying standard input."""
        game.display(state)
        valid_moves = game.actions(state)
        
        if not valid_moves:
            return None
            
        prompt = f"\nPlayer {state.to_move}'s turn ({self.name})\n"
        prompt += f"Valid moves are {valid_moves} (or 'exit' to end game): "
        
        while True:
            move = input(prompt).lower().strip()
            if move in ['exit', 'quit', 'escape']:
                raise GameExit("Game ended by player")
            try:
                move = int(move)
                if move in valid_moves:
                    return move
            except ValueError:
                pass
            print('Invalid move. Try again.')

class RandomPlayer:
    """A player that chooses moves randomly from valid options."""
    def __init__(self, name="Random"):
        self.name = name

    def __call__(self, game, state):
        """Choose a random valid move."""
        valid_moves = game.actions(state)
        if valid_moves:
            return random.choice(valid_moves)
        return None

if PLAY_USER_VS_RANDOM:
    play_mancala(HumanPlayer("Player 1"), RandomPlayer())

# Minimax Player
Now we are going to build an AI player that uses minimax to choose the best move with a variable
number of plies and a utility function we describe to see if it can beat the random player more than 50% of the time.

In [ ]:
def minmax_decision(state, game, depth=5):
    """Given a state in a game, calculate the best move by searching
    forward all the way to the terminal states. [Figure 5.3]"""

    player = game.to_move(state)

    def max_value(state, depth):
        if game.terminal_test(state) or depth == 0:
            return game.utility(state, player)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), depth - 1))
        return v

    def min_value(state, depth):
        if game.terminal_test(state) or depth == 0:
            return game.utility(state, player)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), depth - 1))
        return v

    # Body of minmax_decision:
    return max(game.actions(state), key=lambda a: min_value(game.result(state, a), depth - 1))

In [ ]:
class MinimaxPlayer:
    """A player that uses minimax to choose the best move."""
    
    def __init__(self, name="Minimax"):
        self.name = name

    def __call__(self, game, state):
        """Choose a random valid move."""
        move = minmax_decision(state, game)
        return move
        
if PLAY_MINIMAX_VS_RANDOM:
    play_mancala(MinimaxPlayer(), RandomPlayer())

# Simulating 100 Games(Random player vs Minimax player) using a depth of 5 plies

In [ ]:
minimaxwins = 0
randomwins = 0
ties = 0
for i in range(NUMBER_OF_GAMES_TO_SIMULATE):
    result = play_mancala(MinimaxPlayer(), RandomPlayer())
    if(result == 1):
        minimaxwins += 1
    elif(result == -1):
        randomwins += 1
    else:
        ties += 1

In [ ]:
print("Percentage of games won by the minimax player:", minimaxwins/NUMBER_OF_GAMES_TO_SIMULATE)

# Alpha-Beta Player
Now we are going to build an AI player that uses Alpha-Beta to choose the best move and supports a variable number of plies.

In [ ]:
def alpha_beta_search(state, game, depth):
    """Search game to determine best action; use alpha-beta pruning.
    As in [Figure 5.7], this version searches all the way to the leaves."""

    player = game.to_move(state)

    # Functions used by alpha_beta
    def max_value(state, alpha, beta, depth):
        if game.terminal_test(state) or depth == 0:
            return game.utility(state, player)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), alpha, beta, depth - 1))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta, depth):
        if game.terminal_test(state) or depth == 0:
            return game.utility(state, player)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), alpha, beta, depth - 1))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body of alpha_beta_search:
    best_score = -np.inf
    beta = np.inf
    best_action = None
    for a in game.actions(state):
        v = min_value(game.result(state, a), best_score, beta, depth - 1)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action

In [ ]:
class AlphaBetaPlayer:
    """A player that uses Alpha-Beta to choose the best move."""
    
    def __init__(self, name="AlphaBeta", depth=5):
        self.name = name
        self.depth = depth # depth to go in search

    def __call__(self, game, state):
        """Choose a random valid move."""
        move = alpha_beta_search(state, game, self.depth)
        return move

In [ ]:

if PLAY_ALPHABETA_VS_RANDOM:
    play_mancala(AlphaBetaPlayer(), RandomPlayer())

# Simulating 100 Games(AlphaBeta vs Random) using a depth of 5 plies

In [ ]:
alpha_beta_wins = 0
randomwins = 0
ties = 0
for i in range(NUMBER_OF_GAMES_TO_SIMULATE):
    result = play_mancala(AlphaBetaPlayer(), RandomPlayer())
    if(result == 1):
        alpha_beta_wins += 1
    elif(result == -1):
        randomwins += 1
    else:
        ties += 1

In [ ]:
print("Percentage of games won by the minimax player:", alpha_beta_wins/NUMBER_OF_GAMES_TO_SIMULATE)

# (Extra Credit)
Below is an alphabeta player with 10 plies against the random player.

In [ ]:
alpha_beta_wins = 0
randomwins = 0
ties = 0
for i in range(NUMBER_OF_GAMES_TO_SIMULATE):
    result = play_mancala(AlphaBetaPlayer(depth=10), RandomPlayer())
    if(result == 1):
        alpha_beta_wins += 1
    elif(result == -1):
        randomwins += 1
    else:
        ties += 1

In [ ]:
print("Percentage of games won by the alphabeta with depth 10 player:", alpha_beta_wins/NUMBER_OF_GAMES_TO_SIMULATE)